In [1]:
import pandas as pd
import numpy as np
import emoji
from transformers import AutoTokenizer, AutoModel
from tqdm.notebook import tqdm
import time
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=8)
tqdm.pandas()

from settings import AMBIGUITY_PATH, AMBIGUITY_CLUSTER

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/czestoch/.local/lib/python3.7/site-packages/tqdm/std.py:702: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from src.data.utils import save_to_csv

In [3]:
path = "/scratch/czestoch/sampled_tweets.txt.gz"
tweets = pd.read_csv(path, header=0, lineterminator='\n', encoding='utf-8')['tweet']

In [4]:
len(tweets)

93390

In [5]:
tweets.head()

0    *⃣🆘PLEASE   🆘WE WILL NOT BE SILENT😡plz watch📼 ...
1       *⃣PLEASE   KILL RATE⚠️RESCUE ONLY⚠️Thank you*⃣
2    *⃣💠*⃣💠*⃣💠*⃣💠*⃣💠*⃣💠 How to TRIGGER a LIBERAL wi...
3    "SAFFRON" 🧡Sweet male puppy 3 months 11.5 lbs ...
4          *⃣PLEASE   3 DUMPED IN DROP BOX😡DIES💉2/22*⃣
Name: tweet, dtype: object

In [5]:
# tweets = tweets[:1000]
# len(tweets)

1000

In [7]:
def preprocess_tweets(text):
    return add_spaces_between_emojis(emoji.demojize(text))

def add_spaces_between_emojis(demojified_text):
    new_text = []
    colons = []
    for char in demojified_text:
        if char == ":":
            if colons:
                new_text.append(char + " ")
                colons.pop()
            else:
                colons.append(char)
                new_text.append(" " + char)
        else:
            new_text.append(char)
    return ''.join(new_text)

In [8]:
tweets = tweets.parallel_apply(preprocess_tweets)

In [10]:
all_emojis = pd.read_csv(AMBIGUITY_CLUSTER).emoji.unique()

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")
emojis_in_tokenizer = {}
for em in all_emojis:
    emoji_tensor = tokenizer(emoji.demojize(em), return_tensors='pt')['input_ids']
    emoji_vocab_idx = emoji_tensor[0][1].item()
    # if size is 3 it means emoji token was not splitted so it is a known token,
    # [start] [emoji] [stop]
    # index 3 stands for an unknown token
    if emoji_tensor.size(1) == 3 and emoji_vocab_idx != 3:
        emojis_in_tokenizer[em] = emoji_vocab_idx

print(f"{len(emojis_in_tokenizer)} of our emojis are in this model")
del all_emojis

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/843k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


442 of our emojis are in this model


In [11]:
emojis_in_tokenizer_indices = set(emojis_in_tokenizer.values())
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")
model = AutoModel.from_pretrained("vinai/bertweet-base")

def get_emoji_embedding(text):
    tokenized = np.array(tokenizer.tokenize(text))
    encoded_input = tokenizer(text, return_tensors='pt')
    tokens_ids = encoded_input['input_ids']
    mask = [id_.item() in emojis_in_tokenizer_indices for id_ in tokens_ids[0]]
    if any(mask):
        try:
            features = model(**encoded_input)[0]
        except IndexError:
            return np.nan, np.nan
        return features[0][mask][:].detach().numpy(), tokenized[mask[1:-1]]
    else:
        return np.nan, np.nan

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
out = tweets.progress_apply(get_emoji_embedding)

  0%|          | 0/93390 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (188 > 128). Running this sequence through the model will result in indexing errors


In [13]:
tweets = tweets.to_frame()

In [14]:
tweets[["embedding", "emoji"]] = pd.DataFrame(out.tolist())

In [15]:
tweets.head()

,tweet,embedding,emoji
0,:keycap_asterisk: :SOS_button: PLEASE :SO...,"[[-0.40534008, 0.29607773, 0.18809724, 0.15452...","[:SOS_button:, :SOS_button:, :pouting_face:]"
1,:keycap_asterisk: PLEASE KILL RATE :warning...,NaN,NaN
2,:keycap_asterisk: :diamond_with_a_dot: :key...,NaN,NaN
3,"""SAFFRON"" :orange_heart: Sweet male puppy 3 m...","[[0.13286656, -0.041414626, 0.34103054, 0.1911...","[:orange_heart:, :green_heart:]"
4,:keycap_asterisk: PLEASE 3 DUMPED IN DROP B...,"[[-0.42245123, -0.23179615, -0.060957894, -0.1...","[:pouting_face:, :syringe:]"


In [18]:
tweets = tweets.dropna()

In [22]:
save_to_csv(tweets, "/scratch/czestoch/bert_emojis.csv")

In [39]:
tweets["embedding"] = tweets["embedding"].parallel_apply(lambda x: x.tolist())
tweets["emoji"] = tweets["emoji"].parallel_apply(lambda x: x.tolist())

In [43]:
tweets = tweets.set_index(['tweet']).apply(pd.Series.explode).reset_index()

In [48]:
save_to_csv(tweets, "/scratch/czestoch/bert_emojis.csv")

In [47]:
tweets.groupby("emoji").embedding.count()

emoji
:COOL_button:           115
:Christmas_tree:        353
:FREE_button:           194
:OK_button:             196
:OK_hand:               599
                       ... 
:wrapped_gift:          586
:yellow_heart:          935
:zany_face:            1116
:zipper-mouth_face:      94
:zzz:                   256
Name: embedding, Length: 437, dtype: int64

## Check if unknown tokens are rubbish

In [ ]:
def get_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    features = model(**encoded_input)
    return features[0].detach().cpu().numpy() 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base",\
                                         additional_special_tokens=[":paintbrush_selector:"])
model = AutoModel.from_pretrained("vinai/bertweet-base")
model.resize_token_embeddings(len(tokenizer))

In [ ]:
### Paintbrush is initially not in the vocabulary but was added as an additional token

In [ ]:
print(f"This is an initially known token: {'🖌️' in emojis_in_tokenizer}")
text1 = "This is amazing, trust me! 🖌️"
text2 = "This is absolutely horrible, never ever try doing it 🖌️"
text1 = emoji.demojize(text1)
text2 = emoji.demojize(text2)
one_pass = get_embedding(text1)
second_pass = get_embedding(text2)
(one_pass - second_pass).sum()

In [ ]:
### Heart is initially not in the vocabulary

In [ ]:
print(f"This is an initially known token: {'❤️' in emojis_in_tokenizer}")
text1 = "This is amazing, trust me! ❤️"
text2 = "This is absolutely horrible, never ever try doing it ❤️"
text1 = emoji.demojize(text1)
text2 = emoji.demojize(text2)
one_pass = get_embedding(text1)
second_pass = get_embedding(text2)
(one_pass - second_pass).sum()

In [ ]:
### Emojis below are in the original vocabulary

In [ ]:
print(f"This is an initially known token: {'😂' in emojis_in_tokenizer}")
text1 = "This is amazing, trust me! 😂"
text2 = "This is absolutely horrible, never ever try doing it 😂"
text1 = emoji.demojize(text1)
text2 = emoji.demojize(text2)
one_pass = get_embedding(text1)
second_pass = get_embedding(text2)
(one_pass - second_pass).sum()

In [ ]:
print(f"This is an initially known token: {'💓' in emojis_in_tokenizer}")
text1 = "This is amazing, trust me! 💓"
text2 = "This is absolutely horrible, never ever try doing it 💓"
text1 = emoji.demojize(text1)
text2 = emoji.demojize(text2)
one_pass = get_embedding(text1)
second_pass = get_embedding(text2)
(one_pass - second_pass).sum()

In [ ]:
print(f"This is an initially known token: {'🧡' in emojis_in_tokenizer}")
text1 = "This is amazing, trust me! 🧡"
text2 = "This is absolutely horrible, never ever try doing it 🧡"
text1 = emoji.demojize(text1)
text2 = emoji.demojize(text2)
one_pass = get_embedding(text1)
second_pass = get_embedding(text2)
(one_pass - second_pass).sum()

In [ ]:
### check if embeddinga on unknown tokens that were added to vocabulary are rubish
### check which emojis are in the tokenizer and how to extract their embeddings later
### check parallelization
### save embeddings, yupi